In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [26]:
#moovielens 
movies = pd.read_csv('movies.csv') 
links = pd.read_csv('links.csv')
tags = pd.read_csv('tags.csv')
ratings = pd.read_csv('ratings.csv')

tags['tag'] = tags['tag'].fillna('')

#concatener les tags
movies_with_links = pd.merge(movies, links, on='movieId', how='left')
movie_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# merger les dataframes movies_with_links et movie_tags
movies_with_ratings = pd.merge(movies_with_links, ratings, on='movieId', how='left')

# Calculer la moyenne des ratings
mean_ratings = movies_with_ratings.groupby('movieId')['rating'].mean().reset_index()

# merger les dataframes
data = pd.merge(movies_with_links, movie_tags, on='movieId', how='left')
data = pd.merge(data, mean_ratings, on='movieId', how='left')

#ajouter une colone user_id
data['userId'] = np.random.randint(1, 86537, data.shape[0])

#ajouter une colonne popularity
data['popularity'] = data['rating'].groupby(data['movieId']).transform('count')

data.head()

,movieId,title,genres,imdbId,tmdbId,tag,rating,userId,popularity
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,animation friendship toys animation Disney Pix...,3.893508,55016,1
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,animals based on a book fantasy magic board ga...,3.278179,6130,1
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,sequel moldy old old age old men wedding old p...,3.171271,51696,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,characters chick flick girl movie characters c...,2.868395,58797,1
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,family pregnancy wedding 4th wall aging baby d...,3.076957,85288,1


In [27]:
#nombre de doublons
print(data.duplicated().sum())   #pas de doublos 

# ============= pas de valeurs manquantes ni de doublons =============
data.dropna(subset=['tmdbId'], inplace=True)
#rempcer les valeurs manquantespar ""
data['tag'] = data['tag'].fillna('')
data['rating'] = data['rating'].fillna(data['rating'].mean())

#====netoyyer les colonnes =================================================
data['genres'] = data['genres'].str.replace('|', ' ')
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')
data['title'] = data['title'].str.strip() 
data['tag'] = data['tag'].str.lower()



0


In [31]:
#=================mapping pour le modele NCF=========================
#mapping des userId
user_id_mapping = {id:i for i, id in enumerate(data['userId'].unique())}
data['userId'] = data['userId'].map(user_id_mapping)
#mapping des movieId
movie_id_mapping = {id:i for i, id in enumerate(data['movieId'].unique())}
data['movieId'] = data['movieId'].map(movie_id_mapping)

#==Normaliser les colonnes rating et popularity =======================
scaler = StandardScaler()
data[['rating', 'popularity']] = scaler.fit_transform(data[['rating', 'popularity']])

data.head()

,movieId,title,genres,imdbId,tmdbId,tag,rating,userId,popularity
0,0,Toy Story (1995),Adventure Animation Children Comedy Fantasy,114709,862.0,animation friendship toys animation disney pix...,1.135783,0,0.199106
1,1,Jumanji (1995),Adventure Children Fantasy,113497,8844.0,animals based on a book fantasy magic board ga...,0.342364,1,0.199106
2,2,Grumpier Old Men (1995),Comedy Romance,113228,15602.0,sequel moldy old old age old men wedding old p...,0.204515,2,0.199106
3,3,Waiting to Exhale (1995),Comedy Drama Romance,114885,31357.0,characters chick flick girl movie characters c...,-0.186019,3,0.199106
4,4,Father of the Bride Part II (1995),Comedy,113041,11862.0,family pregnancy wedding 4th wall aging baby d...,0.082905,4,0.199106


In [ ]:
#==========implementer le modele NCF=========================
